# Session 11: Advanced Retrieval with LangChain

## Learning Objectives:

- Understand and implement multiple retrieval strategies for RAG
- Compare naive, BM25, multi-query, parent-document, contextual compression, ensemble, and semantic chunking approaches
- Build RAG chains over a health and wellness knowledge base using LangChain and QDrant

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

---

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

We'll also provide our OpenAI key, as well as our Cohere API key.

> NOTE: Create a `.env` file in this directory with `OPENAI_API_KEY` and `COHERE_API_KEY` to avoid being prompted each time.

In [11]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [12]:
if not os.environ.get("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using our Health and Wellness Guide - a comprehensive resource covering exercise, nutrition, sleep, stress management, habits, and common health concerns.

### Data Preparation

We'll load the wellness guide as a single document, then split it into smaller chunks using a `RecursiveCharacterTextSplitter` for our vector store. We also keep the raw (unsplit) document for use with the Parent Document Retriever and Semantic Chunker later.

In [13]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader("data/HealthWellnessGuide.txt")
raw_docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
wellness_docs = text_splitter.split_documents(raw_docs)

Let's verify our data was loaded and split correctly!

In [14]:
print(f"Raw documents: {len(raw_docs)}")
print(f"Split chunks: {len(wellness_docs)}")
print(f"\nExample chunk:\n{wellness_docs[0]}")

Raw documents: 1
Split chunks: 45

Example chunk:
page_content='The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND MOVEMENT

Chapter 1: Understanding Exercise Basics

Exercise is one of the most important things you can do for your health. Regular physical activity can improve your brain health, help manage weight, reduce the risk of disease, strengthen bones and muscles, and improve your ability to do everyday activities.' metadata={'source': 'data/HealthWellnessGuide.txt'}


## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "wellness_guide".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [15]:
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = QdrantVectorStore.from_documents(
    wellness_docs,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide",
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [16]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [17]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [18]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [19]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [10]:
naive_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

"Exercises that can help with lower back pain include:\n\n- Cat-Cow Stretch: Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening abs and tilting your pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.\n- Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles and raise shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n\nThese exercises are gentle and aimed at stretching and strengthening the lower back muscles to alleviate pain and 

In [11]:
naive_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in maintaining overall health. It supports physical recovery by allowing the body to repair tissues and regenerate, particularly during deep sleep stages. Sleep also consolidates memories and enhances cognitive functions. Additionally, quality sleep helps regulate hormones that control growth and appetite, supports immune system function, and improves mental well-being. A consistent, restful sleep routine contributes to reducing stress, boosting the immune response, and promoting long-term health and happiness.'

In [12]:
naive_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Drinking water and staying hydrated\n- Applying cold or warm compresses to the head or neck\n- Resting in a dark, quiet room\n- Gentle massage of the temples and neck\n- Using peppermint or lavender essential oils\n- Maintaining a regular sleep schedule\n- Practicing deep breathing exercises\n- Engaging in progressive muscle relaxation\n- Taking short walks, preferably in nature\n- Listening to calming music\n\nThese techniques can help alleviate stress and reduce headache symptoms naturally.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [20]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(wellness_docs)

We'll construct the same chain - only changing the retriever.

In [21]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [15]:
bm25_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees. Alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- **Bird Dog:** From hands and knees, extend the opposite arm and leg while keeping your core engaged. Hold each extension for 5 seconds, then switch sides. Aim for 10 repetitions per side.\n- **Pelvic Tilts:** Lie on your back with knees bent. Flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds and repeat 8-12 times.\n\nThese exercises are gentle stretches and strengthening movements that may help alleviate and prevent lower back pain.'

In [16]:
bm25_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a crucial role in overall health. It helps maintain a consistent, high-quality sleep schedule which is essential for physical and mental well-being. Creating an optimal sleep environment—such as keeping the room cool, dark, and quiet, and using comfortable bedding—can improve sleep quality. Practicing good sleep hygiene, like establishing a relaxing bedtime routine and limiting screen time before bed, supports better rest. Adequate sleep contributes to better immune function, mental health, and overall energy levels, thereby positively influencing overall health.'

In [17]:
bm25_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include practicing relaxation techniques such as progressive muscle relaxation, meditation, and deep breathing exercises. Herbal teas like chamomile or valerian root may also help reduce stress and promote relaxation. Additionally, staying well-hydrated, ensuring adequate sleep, managing stress levels, and avoiding known headache triggers can be beneficial. Always consult with a healthcare provider before starting any new supplements or herbal remedies.'

It's not clear that this is better or worse, if only we had a way to test this (SPOILERS: We do, the second half of the notebook will cover this)

### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.

##### Answer:

The example query I tried is `150 minutes per week`. 

Since BM25 looks for the specific term(s) or value within a search query, it generally performs better than embedding based retrieval when we want to look for a specific term or value. Therefore, by looking at the source document, I find this query interesting: "150 minutes per week", which contains a specific value (15) and term (minutes per week). 

BM25 ranks the correct chunk that contains this exact query as the top chunk. By contrast, the embedding retriever ranked it second, while the first chunk it ranks is a semantically related but incorrect "Beginner Weekly Schedule" chunk.

Specific code as below:

In [24]:
query = "150 minutes per week"

print("=== BM25 Results ===")
bm25_docs = bm25_retriever.invoke(query)
for i, doc in enumerate(bm25_docs):
    print(f"\nChunk {i}:\n{doc.page_content[:200]}")

print("\n" + "="*50)

print("=== Embedding Results ===")
embedding_docs = naive_retriever.invoke(query)
for i, doc in enumerate(embedding_docs):
    print(f"\nChunk {i}:\n{doc.page_content[:200]}")


=== BM25 Results ===

Chunk 0:
The four main types of exercise are aerobic (cardio), strength training, flexibility, and balance exercises. A well-rounded fitness routine includes all four types. Adults should aim for at least 150 

Chunk 1:
Types of insomnia:
- Acute insomnia: Short-term, often triggered by stress or life events
- Chronic insomnia: Long-term, occurring at least 3 nights per week for 3 months or more

Natural remedies for

Chunk 2:
Weekly Meal Planning Steps:
1. Review your schedule for the week and identify busy days
2. Choose 3-4 main dishes that can provide leftovers
3. Plan for healthy snacks to avoid impulse eating
4. Creat

Chunk 3:
Adults typically need 7-9 hours of sleep per night. Sleep occurs in cycles of about 90 minutes, alternating between REM (rapid eye movement) and non-REM sleep.

The four stages of sleep:
- Stage 1: Li

=== Embedding Results ===

Chunk 0:
Beginner Weekly Schedule:
- Monday: 20-minute walk + 10 minutes stretching
- Tuesday: 15 minutes b

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [22]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [23]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [24]:
contextual_compression_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help alleviate lower back pain include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees. Arch your back up (like a cat) then let it sag down (like a cow). Perform 10-15 repetitions.\n- **Bird Dog:** From hands and knees, extend your opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- **Pelvic Tilts:** Lie on your back with knees bent. Flatten your lower back against the floor by tightening your abs and tilting your pelvis slightly. Hold for 10 seconds and repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises can help relieve lower back discomfort and prevent future issues.'

In [28]:
contextual_compression_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep significantly impacts overall health by supporting physical repair, mental well-being, and cognitive function. During sleep, your body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Adequate sleep—typically 7-9 hours per night—also helps maintain a healthy immune system, emotional stability, and proper metabolic function. Poor sleep or sleep disorders like insomnia can negatively affect these processes, leading to health issues. Creating an optimal sleep environment can enhance sleep quality and overall health.'

In [29]:
contextual_compression_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for managing stress and headaches include practicing deep breathing exercises, progressive muscle relaxation, and grounding techniques. Taking short walks in nature and listening to calming music can also help reduce stress. For headaches specifically, remedies include staying well-hydrated by drinking water, applying cold or warm compresses to the head or neck, resting in a dark, quiet room, gentle massage of the temples and neck, using peppermint or lavender essential oils, and maintaining a regular sleep schedule.'

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [25]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
) 

In [26]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [32]:
multi_query_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees. Alternate between arching your back up (cat pose) and letting it sag down (cow pose). Perform 10-15 repetitions.\n\n- **Bird Dog:** From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold each extension for about 5 seconds, then switch sides. Aim for 10 repetitions per side.\n\n- **Partial Crunches:** Lie on your back with knees bent, cross arms over your chest, tighten your stomach muscles, and lift your shoulders off the floor. Hold briefly and then lower back down. Do 8-12 repetitions.\n\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat on the floor. Hold the stretch for 15-30 seconds and switch legs.\n\n- **Pelvic Tilts:** Lie on your back with knees bent. Flatten your back against the floor by tightening your abs and tilting your pelvis upward slightly. Hold for about 10 secon

In [33]:
multi_query_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in overall health by supporting physical, mental, and cognitive well-being. During sleep, the body repairs tissues, regulates hormones related to growth and appetite, and consolidates memories, which are essential for healthy functioning. Lack of sufficient quality sleep can lead to issues such as fatigue, immune dysfunction, mental health problems, and impaired cognitive performance. Maintaining good sleep hygiene, such as keeping a consistent schedule, creating a comfortable sleep environment, and practicing relaxation routines, can promote better sleep and thereby enhance overall health.'

In [34]:
multi_query_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Drinking plenty of water to stay hydrated.\n- Applying cold or warm compresses to your head or neck.\n- Resting in a dark, quiet room.\n- Gentle massage of your temples and neck.\n- Using peppermint or lavender essential oils.\n- Maintaining a regular sleep schedule.\n- Practicing immediate stress relief techniques such as deep breathing, progressive muscle relaxation, grounding exercises, taking short walks, or listening to calming music.\n\nThese remedies can help alleviate headache symptoms and reduce stress naturally.'

### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

##### Answer:

Using a single user query to retrieve might miss relevant chunks due to the phrasing/wording of the query (e.g., "workout" vs "exercise"). By contrast, when we generate multiple variations that phrase similar question in different ways, we increase the diversity of the queries that cover different ways of phrasing this similar question semantically. As a result, the union of the chunks that gets retrieved will have a higher chance to cover more ground truth chunks, and this will improve the recall since more relevant chunks get retrieved.

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. We split the full document into large "parent" chunks (e.g. 2000 characters).
2. Each parent chunk is further split into smaller "child" chunks (e.g. 400 characters).
3. The child chunks are stored in a VectorStore, while the parent chunks are stored in an in-memory docstore.
4. When we query our Retriever, we do a similarity search comparing our query vector to the child chunks.
5. Instead of returning the child chunks, we return their associated parent chunks.

The basic idea is:

- **Search** for small, focused chunks (better semantic matching)
- **Return** big chunks (richer surrounding context)

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by defining our parent and child splitters.

In [27]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [28]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="wellness_parent_child",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="wellness_parent_child", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [29]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore=parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [30]:
parent_document_retriever.add_documents(raw_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [31]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [40]:
parent_document_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'To help with lower back pain, some recommended exercises include:\n\n- **Cat-Cow Stretch:** On hands and knees, alternate arching your back up (cat) and sagging it down (cow). Aim for 10-15 repetitions.\n- **Bird Dog:** From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- **Partial Crunches:** Lie on your back with knees bent, cross arms over chest, tighten your stomach muscles, and lift your shoulders off the floor. Do 8-12 repetitions.\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis upward. Hold for 10 seconds. Repeat 8-12 times.\n\nThese exercises can help alleviate discomfort and prevent future episodes of lower back pain. Remember to perfo

In [41]:
parent_document_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep significantly impacts overall health by supporting multiple vital functions. During sleep, the body repairs tissues, balances hormones that regulate growth and appetite, and strengthens the immune system. Adequate sleep (7-9 hours for adults) is essential for mental well-being, cognitive functions like memory consolidation, and emotional regulation. Poor sleep quality or insufficient sleep can lead to issues such as fatigue, decreased concentration, weakened immunity, and increased risk of chronic conditions. Maintaining good sleep hygiene—like having a consistent sleep schedule, creating a relaxing bedtime routine, and optimizing the sleep environment—can improve sleep quality and, in turn, enhance overall health.'

In [42]:
parent_document_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include practicing deep breathing exercises, engaging in mindfulness or meditation, performing gentle stretching or yoga, taking warm baths, listening to calming music, and using essential oils like peppermint or lavender. Additionally, staying well-hydrated by drinking water and maintaining a regular sleep schedule can help manage headaches and reduce stress. Resting in a dark, quiet room and self-massage of the temples and neck are also effective natural approaches.'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [32]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [33]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [47]:
ensemble_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help alleviate lower back pain include:\n\n1. Cat-Cow Stretch: Start on hands and knees, alternate arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n\n2. Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n\n3. Pelvic Tilts: Lie on your back with knees bent, tighten your abs and tilt your pelvis up slightly to flatten your back against the floor. Hold for 10 seconds and repeat 8-12 times.\n\n4. Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles, and raise shoulders off the floor briefly. Do 8-12 repetitions.\n\n5. Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n\nThese exercises are gentle stretches and strengthening movements that can help relieve lower back discomfort and preven

In [48]:
ensemble_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. During sleep, the body repairs tissues, consolidates memories, and regulates hormones that influence growth and appetite. Adequate sleep (7-9 hours per night) supports physical health, mental well-being, and cognitive function. Poor sleep or sleep-related issues like insomnia can lead to problems such as increased stress, impaired immune function, and decreased mental clarity. Creating a healthy sleep environment and practicing good sleep hygiene—such as maintaining a regular sleep schedule, creating a relaxing bedtime routine, and ensuring a comfortable, dark, quiet bedroom—can promote better sleep quality and, consequently, improve overall health.'

In [49]:
ensemble_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Deep breathing exercises (such as inhaling for 4 counts, holding, exhaling, and holding again)\n- Progressive muscle relaxation, tensing and releasing muscle groups\n- Grounding techniques, like naming things you see, hear, feel, smell, and taste\n- Taking short walks, preferably in nature\n- Listening to calming music\n- Engaging in mindfulness and meditation practices\n- Resting in a dark, quiet room\n- Gentle massage of temples and neck\n- Using essential oils like peppermint or lavender\n- Applying cold or warm compresses to the head or neck\n- Staying well-hydrated by drinking water\n- Maintaining a regular sleep schedule\n\nThese approaches can help manage stress and alleviate headache symptoms naturally.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [34]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [35]:
semantic_documents = semantic_chunker.split_documents(raw_docs)

Let's create a new vector store.

In [36]:
semantic_vectorstore = QdrantVectorStore.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide_semantic_chunks"
)

We'll use naive retrieval for this example.

In [37]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [38]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [55]:
semantic_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

"Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n\n- **Partial Crunches:** Lie on your back with knees bent, cross arms over chest, tighten stomach muscles, and raise shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.\n\nAdditionally, starting with gentle movements like walking and stretching, and gradually increasing activity can be beneficial. It's always best to consult with a healthcare provider before starting new exercises, especially if you have ongoing 

In [56]:
semantic_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in overall health by supporting physical restoration, mental well-being, and cognitive function. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Adequate sleep (7-9 hours per night for adults) helps maintain a healthy immune system, reduces the risk of chronic conditions, improves mood, and enhances concentration and learning. Good sleep hygiene and creating an optimal sleep environment contribute significantly to sleep quality, which in turn positively impacts overall health.'

In [57]:
semantic_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

"Some natural remedies for stress include deep breathing exercises, progressive muscle relaxation, grounding techniques (such as naming things you see, hear, feel, smell, and taste), taking short walks in nature, listening to calming music, practicing mindfulness and meditation, and engaging in hobbies. \n\nFor headaches, natural remedies involve staying hydrated by drinking water, applying cold or warm compresses to the head or neck, resting in a dark, quiet room, giving yourself gentle massages of the temples and neck, using peppermint or lavender essential oils, and maintaining a regular sleep schedule. \n\nRemember, it's always best to consult with a healthcare professional if you experience persistent or severe symptoms."

### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?

##### Answer:

When the sentences are short and highly repetitive, the distance metric in between each sentence is very likely to be very small, and therefore when we use semantic chunking (e.g., when using the default 95-th percentile score to split), it's very likely that a high number of sentences ended up in a big chunk.

Some ways to adjust the algorithm:

- Lower the percentile (i.e., the breakpoint threshold) from the default 95-th percentile to something like 80 or even 50-th percentile to creating more chunks. But this may also not lead to very meaningful boundaries between chunks.
- Add a max chunk size to force split
- Revert to the recursive chunking which may better suit this highly repetitive data

---

# 🤝 Breakout Room Part #2

### 🏗️ Activity #1:

Your task is to evaluate the various Retriever methods against each other.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparison between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [2]:
### YOUR CODE HERE
import os
import getpass
from uuid import uuid4


os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")
os.environ["LANGCHAIN_PROJECT"] = f"AIM - Advanced Retrieval - {uuid4().hex[0:8]}"

In [ ]:
## Step 1: Generate the golden dataset

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.testset import TestsetGenerator

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(
    raw_docs,
    testset_size=15
)

df = dataset.to_pandas()
print(f"Total samples: {len(df)}")
print(f"\nDistribution:\n{df['synthesizer_name'].value_counts()}")
df

/var/folders/r4/cqrwpld94ldc4rgfhq1fbx8h0000gn/T/ipykernel_75769/5970106.py:8: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
/var/folders/r4/cqrwpld94ldc4rgfhq1fbx8h0000gn/T/ipykernel_75769/5970106.py:9: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


Applying HeadlinesExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/1 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/5 [00:00<?, ?it/s]

...

Generating personas:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/16 [00:00<?, ?it/s]

Total samples: 16

Distribution:
synthesizer_name
single_hop_specific_query_synthesizer    8
multi_hop_specific_query_synthesizer     8
Name: count, dtype: int64


,user_input,reference_contexts,reference,persona_name,query_style,query_length,synthesizer_name
0,How can I perform a Chest Opener exercise to h...,[PART 1: EXERCISE AND MOVEMENT\n\nChapter 1: U...,To perform a Chest Opener exercise for neck an...,Wellness Enthusiast,PERFECT_GRAMMAR,MEDIUM,single_hop_specific_query_synthesizer
1,How do Cat-Cow Stretch help?,[PART 1: EXERCISE AND MOVEMENT\n\nChapter 1: U...,Cat-Cow Stretch is recommended for lower back ...,Wellness Enthusiast,POOR_GRAMMAR,SHORT,single_hop_specific_query_synthesizer
2,What is insomnia and what are some natural rem...,[PART 2: NUTRITION AND DIET\n\nChapter 4: Fund...,"Insomnia is difficulty falling asleep, staying...",Wellness Enthusiast,WEB_SEARCH_LIKE,SHORT,single_hop_specific_query_synthesizer
3,wht is greek yogrt gud for in a helthy diet an...,[PART 2: NUTRITION AND DIET\n\nChapter 4: Fund...,Greek yogurt is listed as an afternoon snack o...,Wellness Enthusiast,MISSPELLED,LONG,single_hop_specific_query_synthesizer
4,What Chapter 18 say for boost immune?,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,Chapter 18 says to boost immune function you s...,Wellness Enthusiast,POOR_GRAMMAR,SHORT,single_hop_specific_query_synthesizer
5,What Chapter 19 say about how to get better wo...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,Chapter 19 explains that maintaining balance b...,Wellness Enthusiast,POOR_GRAMMAR,LONG,single_hop_specific_query_synthesizer
6,How do I do Meditation for stress? I want to k...,[PART 4: STRESS MANAGEMENT AND MENTAL WELLNESS...,Basic mindfulness meditation steps is: Find a ...,Wellness Enthusiast,POOR_GRAMMAR,MEDIUM,single_hop_specific_query_synthesizer
7,How do you do progressive muscle relaxation fo...,[PART 4: STRESS MANAGEMENT AND MENTAL WELLNESS...,Progressive muscle relaxation is a technique f...,Wellness Enthusiast,POOR_GRAMMAR,LONG,single_hop_specific_query_synthesizer
8,Wut are teh main stratgees for work-life balan...,[<1-hop>\n\nPART 5: BUILDING HEALTHY HABITS Ch...,Chapter 19 recomends several stratgees for wor...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer
9,"According to Chapter 10 and related sections, ...",[<1-hop>\n\nPART 4: STRESS MANAGEMENT AND MENT...,Chapter 10 explains that stress is the body's ...,NaN,NaN,NaN,multi_hop_specific_query_synthesizer


In [41]:
## Step 2: Evaluate the retrievers

import copy, time
from ragas import evaluate, EvaluationDataset, RunConfig
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import (
    LLMContextRecall, Faithfulness, FactualCorrectness,
    ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
)

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
custom_run_config = RunConfig(timeout=360)

retriever_chains = {
    "naive": naive_retrieval_chain,
    "bm25": bm25_retrieval_chain,
    "contextual_compression": contextual_compression_retrieval_chain,
    "multi_query": multi_query_retrieval_chain,
    "parent_document": parent_document_retrieval_chain,
    "ensemble": ensemble_retrieval_chain,
}

all_results = {}

for name, chain in retriever_chains.items():
    print(f"\n{'='*50}")
    print(f"Evaluating: {name}")
    print(f"{'='*50}")

    retriever_dataset = copy.deepcopy(dataset)

    for test_row in retriever_dataset:
        response = chain.invoke({"question": test_row.eval_sample.user_input})
        test_row.eval_sample.response = response["response"].content
        test_row.eval_sample.retrieved_contexts = [
            doc.page_content for doc in response["context"]
        ]
        time.sleep(5)

    ## Avoid the NaN issue for persona_name, query_style, query_length in multi-hop queries
    samples = [test_row.eval_sample for test_row in retriever_dataset]
    eval_dataset = EvaluationDataset(samples=samples)

    result = evaluate(
        dataset=eval_dataset,
        metrics=[
            LLMContextRecall(), Faithfulness(), FactualCorrectness(),
            ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()
        ],
        llm=evaluator_llm,
        run_config=custom_run_config,
    )

    all_results[name] = result
    print(f"{name}: {result}")


/var/folders/r4/cqrwpld94ldc4rgfhq1fbx8h0000gn/T/ipykernel_75769/869665806.py:6: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import (
/var/folders/r4/cqrwpld94ldc4rgfhq1fbx8h0000gn/T/ipykernel_75769/869665806.py:6: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import Faithfulness
  from ragas.metrics import (
/var/folders/r4/cqrwpld94ldc4rgfhq1fbx8h0000gn/T/ipykernel_75769/869665806.py:6: DeprecationWarning: Importing FactualCorrectness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import FactualCorrectness
  from ragas.metrics import (
/v


Evaluating: naive


Evaluating:   0%|          | 0/96 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[59]: TimeoutError()


naive: {'context_recall': 0.9458, 'faithfulness': 0.7686, 'factual_correctness(mode=f1)': 0.6469, 'answer_relevancy': 0.9464, 'context_entity_recall': 0.3357, 'noise_sensitivity(mode=relevant)': 0.0784}

Evaluating: bm25


...

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


parent_document: {'context_recall': 0.9875, 'faithfulness': 0.6668, 'factual_correctness(mode=f1)': 0.6613, 'answer_relevancy': 0.9447, 'context_entity_recall': 0.3413, 'noise_sensitivity(mode=relevant)': 0.1433}

Evaluating: ensemble


Evaluating:   0%|          | 0/96 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[29]: TimeoutError()
Exception raised in Job[35]: TimeoutError()
Exception raised in Job[47]: TimeoutError()
Exception raised in Job[53]: TimeoutError()
Exception raised in Job[59]: TimeoutError()
Exception raised in Job[71]: TimeoutError()
Exception raised in Job[77]: TimeoutError()
Exception raised in Job[83]: TimeoutError()
Exception raised in Job[89]: TimeoutError()
Exception raised in Job[95]: TimeoutError()


ensemble: {'context_recall': 0.9750, 'faithfulness': 0.7018, 'factual_correctness(mode=f1)': 0.6656, 'answer_relevancy': 0.9452, 'context_entity_recall': 0.2381, 'noise_sensitivity(mode=relevant)': 0.1731}


In [47]:
import pandas as pd

comparison = pd.DataFrame({
    name: result.to_pandas().mean(numeric_only=True).to_dict()
    for name, result in all_results.items()
}).T

comparison



,context_recall,faithfulness,factual_correctness(mode=f1),answer_relevancy,context_entity_recall,noise_sensitivity(mode=relevant)
naive,0.945833,0.768603,0.646875,0.946393,0.335747,0.078386
bm25,0.526042,0.542519,0.505625,0.714447,0.184357,0.069874
contextual_compression,0.926042,0.659385,0.719375,0.942020,0.317060,0.050038
multi_query,0.956250,0.679107,0.666250,0.942995,0.268373,0.064683
parent_document,0.987500,0.666752,0.661250,0.944654,0.341270,0.143266
ensemble,0.975000,0.701780,0.665625,0.945188,0.238079,0.173059


In [53]:
from langsmith import Client
import uuid

langsmith_client = Client()

dataset_name = f"Advanced Retrieval - AIE9 - {uuid.uuid4()}"

langsmith_dataset = langsmith_client.create_dataset(
    dataset_name=dataset_name,
    description="Advanced Retrieval Class"
)


In [ ]:
## Step 3: Use langsmith to evaluate the cost and latency
for data_row in dataset.to_pandas().iterrows():
  langsmith_client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

In [60]:

from openevals.llm import create_llm_as_judge
from langsmith.evaluation import evaluate as ls_evaluate

qa_evaluator = create_llm_as_judge(
    prompt=(
        "You are evaluating a QA system. Given the input, assess whether the prediction is correct.\n\n"
        "Input: {inputs}\n"
        "Prediction: {outputs}\n"
        "Reference answer: {reference_outputs}\n\n"
        "Is the prediction correct? Return 1 if correct, 0 if incorrect."
    ),
    feedback_key="qa",
    model="openai:gpt-4.1",
)

labeled_helpfulness_evaluator = create_llm_as_judge(
    prompt=(
        "You are assessing a submission based on the following criterion:\n\n"
        "helpfulness: Is this submission helpful to the user, "
        "taking into account the correct reference answer?\n\n"
        "Input: {inputs}\n"
        "Submission: {outputs}\n"
        "Reference answer: {reference_outputs}\n\n"
        "Does the submission meet the criterion? Return 1 if yes, 0 if no."
    ),
    feedback_key="helpfulness",
    model="openai:gpt-4.1",
)

def make_chain_target(chain):
    """Wrap chain.invoke so it returns just the answer string."""
    def target(inputs):
        result = chain.invoke(inputs)
        return result["response"].content
    return target

for name, chain in retriever_chains.items():
    print(f"\nRunning LangSmith evaluation for: {name}")
    ls_evaluate(
        make_chain_target(chain),
        data=dataset_name,
        evaluators=[qa_evaluator, labeled_helpfulness_evaluator],
        metadata={"revision_id": name},
        experiment_prefix=name,
    )



Running LangSmith evaluation for: naive
View the evaluation results for experiment: 'naive-b3b5ef56' at:
https://smith.langchain.com/o/991456bb-c3e2-422e-957d-4e5bfe176b65/datasets/700c19dd-3cb8-4710-94e7-cc8cff81a136/compare?selectedSessions=949a2a21-86d1-43d0-8b8d-523e6fd90e61




0it [00:00, ?it/s]


Running LangSmith evaluation for: bm25
View the evaluation results for experiment: 'bm25-59c47242' at:
https://smith.langchain.com/o/991456bb-c3e2-422e-957d-4e5bfe176b65/datasets/700c19dd-3cb8-4710-94e7-cc8cff81a136/compare?selectedSessions=980593bf-9d80-42a0-9571-fdcf5b5fbdba




0it [00:00, ?it/s]


Running LangSmith evaluation for: contextual_compression
View the evaluation results for experiment: 'contextual_compression-c4b42579' at:
https://smith.langchain.com/o/991456bb-c3e2-422e-957d-4e5bfe176b65/datasets/700c19dd-3cb8-4710-94e7-cc8cff81a136/compare?selectedSessions=753e4491-992f-4a76-b1b1-82ba4215c93d




...

0it [00:00, ?it/s]


Running LangSmith evaluation for: parent_document
View the evaluation results for experiment: 'parent_document-b8f22c56' at:
https://smith.langchain.com/o/991456bb-c3e2-422e-957d-4e5bfe176b65/datasets/700c19dd-3cb8-4710-94e7-cc8cff81a136/compare?selectedSessions=6dcea39f-06be-4ee7-aba4-b761e53bec28




0it [00:00, ?it/s]


Running LangSmith evaluation for: ensemble
View the evaluation results for experiment: 'ensemble-a775755a' at:
https://smith.langchain.com/o/991456bb-c3e2-422e-957d-4e5bfe176b65/datasets/700c19dd-3cb8-4710-94e7-cc8cff81a136/compare?selectedSessions=3e11d5b0-3a07-4fde-89a6-3000dbbc5a8f




0it [00:00, ?it/s]

## Metrics comparison of these retrievers


In [62]:
# Compare the retriever performance metrics

import pandas as pd

comparison = pd.DataFrame({
    name: result.to_pandas().mean(numeric_only=True).to_dict()
    for name, result in all_results.items()
}).T

comparison



,context_recall,faithfulness,factual_correctness(mode=f1),answer_relevancy,context_entity_recall,noise_sensitivity(mode=relevant)
naive,0.945833,0.768603,0.646875,0.946393,0.335747,0.078386
bm25,0.526042,0.542519,0.505625,0.714447,0.184357,0.069874
contextual_compression,0.926042,0.659385,0.719375,0.942020,0.317060,0.050038
multi_query,0.956250,0.679107,0.666250,0.942995,0.268373,0.064683
parent_document,0.987500,0.666752,0.661250,0.944654,0.341270,0.143266
ensemble,0.975000,0.701780,0.665625,0.945188,0.238079,0.173059


## Compare the latency and cost of these retrievers using langsmith

Here's the langsmith trace that compares these 6 retriever strategies: [https://smith.langchain.com/public/46f43d3d-779e-4c6d-96b3-e85177292ed2/d](https://smith.langchain.com/public/46f43d3d-779e-4c6d-96b3-e85177292ed2/d)

Comparison of the latency by retriever:
![Latency Comparison by Retriever](latency_retriever.png)


Comparison of the cost by retriever:
![Cost Comparison by Retriever](cost_retriever.png)



## Analysis and Conclusion on Retrieval Methods

### Results Summary

| Retriever | Cost | P50 Latency | Context Recall | Faithfulness | Factual Correctness | Answer Relevancy | Context Entity Recall | Noise Sensitivity |
|-----------|------|-------------|---------------|-------------|--------------------|-----------------|--------------------|------------------|
| naive | ~$0.0035 | ~2.1s | 0.946 | 0.769 | 0.647 | 0.946 | 0.336 | 0.078 |
| bm25 | ~$0.0018 | ~1.2s | 0.526 | 0.543 | 0.506 | 0.714 | 0.184 | 0.070 |
| contextual_compression | ~$0.002 | ~1.8s | 0.926 | 0.659 | 0.719 | 0.942 | 0.317 | 0.050 |
| multi_query | ~$0.0047 | ~3.1s | 0.956 | 0.679 | 0.666 | 0.943 | 0.268 | 0.065 |
| parent_document | ~$0.0029 | ~1.8s | 0.988 | 0.667 | 0.661 | 0.945 | 0.341 | 0.143 |
| ensemble | ~$0.0063 | ~3.9s | 0.975 | 0.702 | 0.666 | 0.945 | 0.238 | 0.173 |

### Conclusion

For this wellness dataset, the **naive retriever** actually provides great overall balance of performance, cost, and latency. Performance wise, it has great context recall score (0.95), highest faithfulness score (0.77), high answer relevancy (0.95), and other scores are all decently well. Cost/latency wise, it has a moderate cost and latency. For example, although the cost is higher than bm25, the performance is much better; although ensemble has twice the cost than naive, the marginal performance gain of ensemble is actually minimal on this golden dataset. Same trend for latency as well.

**Parent document** is a strong alternative which I may also choose instead of naive retriever. It has the highest context recall score (0.99), and comparable scores to the naive retriever in other metrics except faithfulness and noise sensitivity. The relatively high noise sensitivity (0.14) is because the large parent chunks could contain irrelevant information. Faithfulness is lower also likely because parent document contains more irrelevant information which caused faithfuless score to be lower. But I think these metrics has smaller effect in actual user experience compared to other metrics. On the other hand, parent document has lower cost and latency than naive, which makes it strong alternative to the native method. 

So for this dataset, I would choose either naive retriever or parent document. But since parent document has lower cost/latency with comparable if not better metrics than naive, I would likely start with parent document approach. We can iterate and change the retrieval strategy based on actual user feedback.

#### Appendix: analysis of other retrievers

**Contextual compression** is a good alternative — it achieves the best factual correctness (0.72) and lowest noise sensitivity (0.05), meaning it retrieves the most precise context. Its cost and latency are also decent, although the cost from Langsmith may not be accurate since we need to do re-rank using Cohere, which might incur additional cost.

**Ensemble and multi_query** offer marginal improvements over naive/parent document but at significantly higher cost (1.3-1.8x) and latency (1.5-1.9x), making them poor value tradeoffs. For multi-query, this is likely because we introduce extra LLM calls to create additional queries, while ensemble executes all the retrieval strategies, all of which introduce much higher cost and latency. 

**BM25** is clearly the weakest for this golden dataset with larger performance gap (e.g., context recall 0.53 vs 0.95+), although it is the cheapest and fastest. BM25 makes more sense if the user query contains more term specific items.

To me the key takeaway is that since our source document is a single, well-structured document with clear topical sections, simple naive retrieval works decently well. As our text corpus becomes more complicated like having various different documents across various domains/topics, more advanced strategies like ensemble and multi_query may show greater benefits.


### Class notes:


#### Golden dataset 
- How many test samples are need
    - 10-15 samples
- Whether to use unrolled or abstract
- Distrubtion of samples
- which LLM to use
    - consider cost, latency, performance
- human vs synthetic generated data

#### Retriever
- determine the number of docs/chunks to retrieve
- parent/child document chunk size

#### Metrics 
- Determine which ragas metrics to use
- how many runs to evaluate 
- how to measure latency (across a few runs) and cost (embedding cost, LLM cost)
- Is faster retrieval better or more accurate retrieval better, what tradeoffs are you willing to pay (more cost for better performance or lower cost for faster performance)